In [ ]:
# torch imports
import torch
# Misc imports
import os 
import sys
import seaborn as sns
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSeg')
# Ionpy imports
from ionpy.util import Config
from ionpy.analysis import ResultsLoader

# Define some useful paths.
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
       '/storage'
))
# Set some defaults
rs = ResultsLoader()
sns.set_style("darkgrid")
torch.set_printoptions(linewidth=200)

# For using code without restarting.
%load_ext autoreload
%autoreload 2
# For using yaml configs.
%load_ext yamlmagic

In [ ]:
%%yaml default_cfg 

experiment:
    exp_root: '?'
    inference_seed: 40
    hard_pred_threshold: 0.5 

log:
    root: '?'
    log_interval: 10 
    log_image_stats: True 
    log_pixel_stats: False 
    gether_inference_stats: True
    summary_compute_global_metrics: False 

dataloader:
    batch_size: '?'
    num_workers: 1 
    pin_memory: True 

In [ ]:
%%yaml calibration_cfg 

calibrator:
    _name: None 

local_calibration:
    num_prob_bins: 15
    neighborhood_width: 3

global_calibration:
    num_classes: '?' 
    num_prob_bins: 15
    neighborhood_width: 3

In [ ]:
%%yaml model_cfg 

# For standard datasets
#####################################
data:
    preload: False 

model:
    pred_label: '?' 
    _type: "standard"
    pretrained_exp_root : None
    ## Use For Uncalibrated
    # checkpoint: 'max-val-dice_score'
    ## Use For Calibrated 
    checkpoint: 'min-val-ece_loss'

## Gather Inference Options.

In [ ]:
%%yaml experiment_cfg 

# Use this field if you want to call it something else beyond the base_model_dir.
name: "ACDC_Experiment_1_ALLSPLITS"


base_model: 
    - "/storage/vbutoi/scratch/ESE/training/07_09_24_ACDC_PixelCELoss/20240709_120223-2YG5-4666f171cc2ff0127caa727844be2be1"
    - "/storage/vbutoi/scratch/ESE/training/07_09_24_ACDC_SoftDiceLoss/20240709_115941-2WD3-0a311a8965d658cbf9bcf19bfaedda25"
    - "/storage/vbutoi/scratch/ESE/training/07_09_24_ACDC_PixelCELoss_wSVLS/20240709_120633-QSUE-402c2b7fcaf6f1bdbd5a96f7547d713d"
    - "/storage/vbutoi/scratch/ESE/training/07_09_24_ACDC_SoftDiceLoss_wSVLS/20240709_120353-5EEZ-f10c991a61dc710b092da75ef9b15573"
    - "/storage/vbutoi/scratch/ESE/calibration/07_12_24_ACDC_CalibratorSet_Experiment1"

data:
    _class: "ese.experiment.datasets.ACDC"
    split: ("cal", "val")
    label_threshold: null # Need to compare with non-noisy gt.

dataloader:
    batch_size: 8

In [ ]:
# Local imports
from ese.experiment.analysis.analysis_utils.submit_utils import get_ese_inference_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([calibration_cfg, model_cfg])
# Get the different experiment cfg yamls.
inf_cfgs = get_ese_inference_configs(
    exp_cfg=experiment_cfg, 
    base_cfg=base_cfg
)

In [ ]:
len(inf_cfgs)

## Running Jobs

In [ ]:
from ese.experiment.analysis.run_inference import get_cal_stats

In [ ]:
from ese.experiment.experiment import run_ese_exp

###### Run individual jobs
run_ese_exp(
    config=inf_cfgs[-8], 
    job_func=get_cal_stats,
    run_name='debug',
    # run_name=exp_name,
    show_examples=True,
    gpu='0',
    # gpu='4',
) 

In [ ]:
# from ese.experiment.experiment import submit_ese_exps 

# #### Run Batch Jobs
# submit_ese_exps(
#     config_list=inf_cfgs,
#     job_func=get_cal_stats,
#     available_gpus=['0', '1', '2', '3']
#     # available_gpus=['4', '5', '6', '7']
# )